# Get realtime models working

In [1]:
# Autoreload possibly interferes with IntelliJ debugging
# %reload_ext autoreload
# %autoreload 2
import logging

from models.realtime.realtime import mne_from_buffer

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: logging.info(msg)

In [2]:
import numpy as np
import mne
from realtime import mne_from_buffer

buffer = np.zeros((1, 7501), dtype=np.float32)
ch_names = ['Fpz-M1']

raw, mne_filtered = mne_from_buffer(buffer, ch_names, 250)

Creating RawArray with float64 data, n_channels=1, n_times=7501
    Range : 0 ... 7500 =      0.000 ...    30.000 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2751 samples (11.004 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower tr

In [6]:
# import io
# import sys
# import contextlib
# from convert import get_filtered_and_scaled_data
# import mne
# 
# # Create a buffer to capture both stdout and stderr
# output_buffer = io.StringIO()
# 
# # Redirect both stdout and stderr to the buffer
# with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
#     # Run the function that produces output and warnings
#     mne_filtered = get_filtered_and_scaled_data(raw)
# 
# # Display all the captured output in one block
# print(output_buffer.getvalue())


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 2751 samples (11.004 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)




In [10]:
# df = mne_filtered.get_data(picks='Fpz-M1', units=dict(eeg="uV"), return_times=True)
# df[1]

array([0.0000e+00, 4.0000e-03, 8.0000e-03, ..., 2.9992e+01, 2.9996e+01,
       3.0000e+01])

In [3]:
import yasa_features

channels = raw.info['ch_names']
sfreq = raw.info['sfreq']

yasa_feats, channel_feats_dict = yasa_features.extract_yasa_features2(log, channels, mne_filtered)


C:\Users\graha\.conda\envs\tf\lib\site-packages\antropy\entropy.py:1012: RuntimeWarning: invalid value encountered in divide
  mob = np.sqrt(dx_var / x_var)
C:\Users\graha\.conda\envs\tf\lib\site-packages\antropy\entropy.py:1013: RuntimeWarning: invalid value encountered in divide
  com = np.sqrt(ddx_var / dx_var) / mob
C:\Users\graha\.conda\envs\tf\lib\site-packages\yasa\spectral.py:358: RuntimeWarning: invalid value encountered in divide
  bp /= total_power
C:\dev\play\brainwave-processor\yasa_features.py:114: RuntimeWarning: invalid value encountered in divide
  spectral_centroid = np.sum(freqs * psd, axis=1) / np.sum(psd, axis=1)
C:\Users\graha\.conda\envs\tf\lib\site-packages\antropy\entropy.py:253: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / psd.sum(axis=axis, keepdims=True)
C:\Users\graha\.conda\envs\tf\lib\site-packages\antropy\entropy.py:366: RuntimeWarning: invalid value encountered in divide
  W /= sum(W)
C:\Users\graha\.conda\envs\tf\lib\site-pack

In [4]:
yasa_feats_orig = yasa_feats.copy()

In [5]:
yasa_feats = yasa_feats_orig.copy()

In [46]:
# # Drop columns containing 'c7' or 'p2' in their names
# columns_to_drop = yasa_feats.filter(regex='c7|p2').columns
# yasa_feats.drop(columns=columns_to_drop, inplace=True)

In [6]:
yasa_feats.transpose()

epoch,0
Fpz-M1_eeg_abspow,0.0
Fpz-M1_eeg_abspow_c7min_norm,0.0
Fpz-M1_eeg_abspow_p2min_norm,0.0
Fpz-M1_eeg_alpha,NaN
Fpz-M1_eeg_alpha_c7min_norm,NaN
...,...
Fpz-M1_eeg_thetaabsaa_c7min_norm,0.0
Fpz-M1_eeg_thetaabsaa_p2min_norm,0.0
Fpz-M1_eeg_thetaabsab,0.0
Fpz-M1_eeg_thetaabsab_c7min_norm,0.0


In [48]:
import os
import pandas as pd
import scaling

input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")
# Find Brainflow files that haven't been copied over

scale_by_stats = scaling.scale_by_stats(yasa_feats, stats_df)
yasa_feats = yasa_feats.join(scale_by_stats.add_suffix('_s'))


In [49]:
yasa_feats.transpose()

epoch,0
Fpz-M1_eeg_abspow,0.000000
Fpz-M1_eeg_alpha,NaN
Fpz-M1_eeg_alphaaa,NaN
Fpz-M1_eeg_alphaab,NaN
Fpz-M1_eeg_alphaabs,0.000000
...,...
Fpz-M1_eeg_thetaaa_s,NaN
Fpz-M1_eeg_thetaab_s,NaN
Fpz-M1_eeg_thetaabs_s,-0.075955
Fpz-M1_eeg_thetaabsaa_s,-0.111960


In [50]:
yasa_df = yasa_feats
yasa_df['60MinsBeforeReadyToSleep'] = 1 

In [51]:
# Rename columns that start with 'Fpz-M1' to 'Main'
yasa_df.rename(columns=lambda x: x.replace('Fpz-M1', 'Main') if x.startswith('Fpz-M1') else x, inplace=True)
# Keep only columns that end with '_s' and the '60MinsBeforeReadyToSleep' column
# columns_to_keep = yasa_df.filter(regex='_s$').columns.tolist()
# columns_to_keep.append('60MinsBeforeReadyToSleep')
# yasa_df = yasa_df[columns_to_keep]

In [52]:
yasa_df.transpose()

epoch,0
Main_eeg_abspow,0.000000
Main_eeg_alpha,NaN
Main_eeg_alphaaa,NaN
Main_eeg_alphaab,NaN
Main_eeg_alphaabs,0.000000
...,...
Main_eeg_thetaab_s,NaN
Main_eeg_thetaabs_s,-0.075955
Main_eeg_thetaabsaa_s,-0.111960
Main_eeg_thetaabsab_s,-0.025055


In [11]:
import realtime

tired_vs_wired_prediction = realtime.run_tired_vs_wired(yasa_df)
tired_vs_wired_prediction

AttributeError: module 'realtime' has no attribute 'run_tired_vs_wired'